<a href="https://colab.research.google.com/github/minidiablo05/PR/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install optuna
!pip install torchmetrics
import optuna
import torch
import numpy as np
import sklearn
import torchmetrics
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

torch.manual_seed(42)

In [9]:
# Важно, чтобы исполнение программы роисходило с использованием cuda

if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
elif torch.cpu.is_available():
    device = 'cpu'
device

'cuda'

In [10]:
# выгружаем датасет для дольнейшего анализа
heart = pd.read_csv('heart.csv')
df = heart.copy()
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [12]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [13]:
df.isnull().sum()

,0
Age,0
Sex,0
ChestPainType,0
RestingBP,0
Cholesterol,0
FastingBS,0
RestingECG,0
MaxHR,0
ExerciseAngina,0
Oldpeak,0


In [14]:
# Убираем колонки, с лишней информацией

df = df.drop(columns=['ChestPainType', 'ExerciseAngina', 'HeartDisease'])

In [15]:
replace_dict = {
    'Sex': {'F': 0, 'M': 1},
    'RestingECG': {'Normal': 0, 'ST': 1, 'LVH': 2},
    'ST_Slope': {'Up': 0, 'Flat': 1, 'Down': 2},
}

df = df.replace(replace_dict)

/tmp/ipython-input-282323292.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(replace_dict)


In [16]:
df.head()

,Age,Sex,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,Oldpeak,ST_Slope
0,40,1,140,289,0,0,172,0.0,0
1,49,0,160,180,0,0,156,1.0,1
2,37,1,130,283,0,1,98,0.0,0
3,48,0,138,214,0,0,108,1.5,1
4,54,1,150,195,0,0,122,0.0,0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          918 non-null    int64  
 1   Sex          918 non-null    int64  
 2   RestingBP    918 non-null    int64  
 3   Cholesterol  918 non-null    int64  
 4   FastingBS    918 non-null    int64  
 5   RestingECG   918 non-null    int64  
 6   MaxHR        918 non-null    int64  
 7   Oldpeak      918 non-null    float64
 8   ST_Slope     918 non-null    int64  
dtypes: float64(1), int64(8)
memory usage: 64.7 KB


In [18]:
def train2(model, optimizer, criterion, metric, train_loader, valid_loader,
               n_epochs):
    history = {"train_losses": [], "train_metrics": [], "valid_metrics": []}
    for epoch in range(n_epochs):
        total_loss = 0.
        metric.reset()
        for X_batch, y_batch in train_loader:
            model.train()
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            metric.update(y_pred, y_batch)
        mean_loss = total_loss / len(train_loader)
        history["train_losses"].append(mean_loss)
        history["train_metrics"].append(metric.compute().item())
        history["valid_metrics"].append(
            evaluate_tm(model, valid_loader, metric).item())
        print(f"Epoch {epoch + 1}/{n_epochs}, "
              f"train loss: {history['train_losses'][-1]:.4f}, "
              f"train metric: {history['train_metrics'][-1]:.4f}, "
              f"valid metric: {history['valid_metrics'][-1]:.4f}")
    return history


In [19]:
def evaluate_tm(model, data_loader, metric):
    model.eval()
    metric.reset()  # reset the metric at the beginning
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            metric.update(y_pred, y_batch)
    return metric.compute()

In [20]:
class CholestirinClassifier(nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(n_inputs, n_hidden1),
            nn.ReLU(),
            nn.Linear(n_hidden1, n_hidden2),
            nn.ReLU(),
            nn.Linear(n_hidden2, 1)
        )

    def forward(self, X):
        return self.mlp(X)

In [21]:
# df = df.iloc[:len(df)]
X = df.drop(columns=['Cholesterol'])
y = df['Cholesterol']

# Разделяем данные на несколько выборок для обучения и тестирования
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.35, random_state=42)

# Нормализуем данные для повышения точности работы модели.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train.values).unsqueeze(1))
valid_dataset = TensorDataset(torch.FloatTensor(X_valid), torch.FloatTensor(y_valid.values).unsqueeze(1))
test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test.values).unsqueeze(1))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [22]:
accuracy = torchmetrics.MeanAbsoluteError().to(device)
loss = nn.MSELoss()

In [23]:
def objective(trial):
    learning_rate = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    n_hidden1 = trial.suggest_int("n_hidden1", 8, 256)
    n_hidden2 = trial.suggest_int("n_hidden2", 8, 256)
    model = CholestirinClassifier(X_train.shape[1], n_hidden1, n_hidden2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    history = train2(model, optimizer, loss, accuracy, train_loader, valid_loader, 10)
    validation_accuracy = min(history["valid_metrics"])
    return validation_accuracy


In [24]:
# ищи оптимальные параметры
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

def stop_condition(study, trial):
  if study.best_value <= 1.3:
      study.stop()

study.optimize(objective, callbacks=[stop_condition], n_trials=2)

[I 2025-10-24 06:08:41,479] A new study created in memory with name: no-name-802c59bd-e9a0-48ef-9fca-d7f4d4ee6cd4


Epoch 1/10, train loss: 51978.4536, train metric: 200.0726, valid metric: 195.9379
Epoch 2/10, train loss: 51453.2935, train metric: 199.2921, valid metric: 194.9423
Epoch 3/10, train loss: 50751.0093, train metric: 198.1350, valid metric: 193.5132
Epoch 4/10, train loss: 49609.2385, train metric: 196.5500, valid metric: 191.4995
Epoch 5/10, train loss: 48255.8796, train metric: 194.2549, valid metric: 188.7834
Epoch 6/10, train loss: 46527.4561, train metric: 191.2695, valid metric: 185.2602


[I 2025-10-24 06:08:42,558] Trial 0 finished with value: 160.81698608398438 and parameters: {'lr': 0.00031489116479568613, 'n_hidden1': 244, 'n_hidden2': 190}. Best is trial 0 with value: 160.81698608398438.


Epoch 7/10, train loss: 44107.0009, train metric: 187.4881, valid metric: 180.7053
Epoch 8/10, train loss: 41522.4883, train metric: 182.5939, valid metric: 175.2370
Epoch 9/10, train loss: 38419.4226, train metric: 176.7295, valid metric: 168.5928
Epoch 10/10, train loss: 35048.9292, train metric: 169.7279, valid metric: 160.8170
Epoch 1/10, train loss: 51812.7800, train metric: 199.9038, valid metric: 195.5304
Epoch 2/10, train loss: 50822.3533, train metric: 198.1042, valid metric: 192.5672
Epoch 3/10, train loss: 48124.1235, train metric: 193.6438, valid metric: 185.6724
Epoch 4/10, train loss: 42689.1691, train metric: 183.8297, valid metric: 172.0853
Epoch 5/10, train loss: 34306.8018, train metric: 166.0800, valid metric: 149.0235
Epoch 6/10, train loss: 23797.0206, train metric: 137.4324, valid metric: 117.5235
Epoch 7/10, train loss: 15257.6459, train metric: 106.5314, valid metric: 92.9282


[I 2025-10-24 06:08:42,922] Trial 1 finished with value: 83.19534301757812 and parameters: {'lr': 0.0024810409748678114, 'n_hidden1': 46, 'n_hidden2': 46}. Best is trial 1 with value: 83.19534301757812.


Epoch 8/10, train loss: 11660.9637, train metric: 87.6948, valid metric: 85.9891
Epoch 9/10, train loss: 11101.1218, train metric: 82.7748, valid metric: 84.4554
Epoch 10/10, train loss: 10769.6210, train metric: 81.1976, valid metric: 83.1953


Можно заметить аномальное поведение мадели, которое подтверждает нашу гипотизу
Обучем модель на лучшем пораметре и протестируем.

In [25]:
# Обучаем модель по найденным параметрам
model = CholestirinClassifier(n_inputs=X_train.shape[1],
                     n_hidden1=study.best_params['n_hidden1'],
                     n_hidden2=study.best_params['n_hidden2']).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=study.best_params['lr'])
_ = train2(model, optimizer, loss, accuracy, train_loader, valid_loader,
           n_epochs=20)

Epoch 1/20, train loss: 51871.3579, train metric: 199.9862, valid metric: 195.7137
Epoch 2/20, train loss: 50945.3315, train metric: 198.6521, valid metric: 193.3476
Epoch 3/20, train loss: 48680.6189, train metric: 195.0022, valid metric: 187.5991
Epoch 4/20, train loss: 44126.3828, train metric: 186.9536, valid metric: 175.9914
Epoch 5/20, train loss: 36364.7345, train metric: 171.8988, valid metric: 155.3560
Epoch 6/20, train loss: 25955.3018, train metric: 146.0747, valid metric: 125.4938
Epoch 7/20, train loss: 16429.7757, train metric: 112.5448, valid metric: 97.3636
Epoch 8/20, train loss: 12306.2892, train metric: 91.4918, valid metric: 88.4556
Epoch 9/20, train loss: 11443.2707, train metric: 84.8550, valid metric: 86.8710
Epoch 10/20, train loss: 11098.7924, train metric: 83.3540, valid metric: 85.4622
Epoch 11/20, train loss: 10867.3025, train metric: 82.2974, valid metric: 84.2105
Epoch 12/20, train loss: 10602.8483, train metric: 81.1846, valid metric: 83.3911
Epoch 13/20,

In [26]:

# Выводим несколько пробных запросов дабы дать пример точности
model.eval()
with torch.no_grad():
    indices = torch.randperm(len(test_dataset))[:5]

    for idx in indices:
        X_sample, y_actual = test_dataset[idx]
        X_sample = X_sample.to(device)
        y_pred = model(X_sample)
        print(f"Prediction: {y_pred.item() :.1f} Actual: {y_actual.item() :.1f}")

Prediction: 180.8 Actual: 289.0
Prediction: 208.8 Actual: 0.0
Prediction: 278.4 Actual: 186.0
Prediction: 326.4 Actual: 288.0
Prediction: 155.9 Actual: 237.0


Модель не только не может точно спрогназировать результат, но и выдаёт ошибки.